### Imports

In [ ]:
#Libraries
from WebScraperPageCount import WebScraper
import pandas as pd
import os
import fnmatch
import tempfile
import psycopg2
from sqlalchemy import create_engine

os.environ['MPLCONFIGDIR'] = "/home/jovyan"
import matplotlib.pyplot as plt



## 1. Daten scrapen, Dataframe erstellen

In [ ]:
    dfImmo = WebScraper().runImmo()
    dfHome = WebScraper().runHome()

    data = pd.concat([dfHome, dfImmo])
    data.to_excel('Output.xlsx', index=False)

## 2. Datenbereinigung

In [42]:
data.head()


,Number of Rooms,Size [m²],Price [CHF],Street,Zip Code,City,State
1,100,4.5,1590,Via Ronco 2,6883,Novazzano,NaN
3,138,4.5,1490,Klosterstieg 8,8240,Thayngen,NaN
4,80,3.5,1450,Via Bellinzona 1a,6512,Giubiasco,NaN
5,None,5,1650,Lärchenweg 1,3363,Oberönz,NaN
6,110,5.5,2100,Industriestrasse 17,5033,Buchs AG,NaN


In [54]:
data.describe()

,Number of Rooms,Size [m²],Price [CHF],Street,Zip Code,City,State
count,7359,7359,7359,7359,7359,7359,943
unique,253,180,907,6014,1334,1404,25
top,None,3.5,1650,Seestrasse 119/121,9000,Basel,Bern
freq,927,1847,100,13,139,342,236


#### 2.1 Leere und falsche Adresseinträge auf NaN setzen

In [43]:
import numpy as np

data['Street'].isnull().sum()
data_work = data
data_work['Street'].replace('', np.nan, inplace=True)
data_work['Street'].replace('None', np.nan, inplace=True)
data_work['Street'].replace('.', np.nan, inplace=True)
data_work['Street'].replace('44', np.nan, inplace=True)
data_work['Street'].replace('70', np.nan, inplace=True)
data_work['Price [CHF]'].replace('None', np.nan, inplace=True)

In [44]:
data_work.describe()

,Number of Rooms,Size [m²],Price [CHF],Street,Zip Code,City,State
count,7359,7359,7359,7359,7359,7359,943
unique,253,180,907,6014,1334,1404,25
top,None,3.5,1650,Seestrasse 119/121,9000,Basel,Bern
freq,927,1847,100,13,139,342,236


#### 2.2 Nan Einträge löschen

In [45]:
data_work.dropna(subset=['Street','Price [CHF]'], inplace=True)

In [46]:
data_work.describe()

,Number of Rooms,Size [m²],Price [CHF],Street,Zip Code,City,State
count,7359,7359,7359,7359,7359,7359,943
unique,253,180,907,6014,1334,1404,25
top,None,3.5,1650,Seestrasse 119/121,9000,Basel,Bern
freq,927,1847,100,13,139,342,236


In [ ]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(data_work)

#### 2.3 Duplikate anhand von Adresse und Preis entfernen

In [58]:
data_cleansed = data_work.drop_duplicates(subset=['Price [CHF]', 'Street'])
data_cleansed.describe()

,Number of Rooms,Size [m²],Price [CHF],Street,Zip Code,City,State
count,6886,6886,6886,6886,6886,6886,587
unique,253,155,907,6014,1334,1402,24
top,None,3.5,1650,Vy d'Etra 35,6900,Basel,Bern
freq,909,1807,95,11,128,327,171
